# Red Neuronal

## Librerias

In [ ]:
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import re
import unicodedata
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sqlalchemy import create_engine
import json
import psycopg2
from psycopg2.extras import execute_batch

## Cargar Modelo y Utilidades

In [ ]:
# -------------------
# 1. Cargar modelo y utilidades
# -------------------
model = tf.keras.models.load_model("modelo_categorias.keras")

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("labelencoder.pkl", "rb") as f:
    le = pickle.load(f)

## Normaliza Texto

In [ ]:
# -------------------
# 2. Función de normalización (misma que en el entrenamiento)
# -------------------
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

## CArgar Datos

In [ ]:
def conection_bd():
    try:
        host = "directob2b.postgres.database.azure.com"
        port = "5432"
        database = "ecommerce"
        user = "directob2b"
        password = "Noctua2025$"
        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
        print("Conexión exitosa 🚀")
        return engine
    except Exception as e:
        print("Error al conectar con SQLAlchemy:", e)
        return None

In [ ]:
engine = conection_bd()
try:
    query = """SELECT tp.csku,
       tp.cnombre,
       tp.cdescripcion,
       tp.cmarca,
       tnc.id_subcategoria as id_subcategoria,
       tnc.nombre_subcategoria as subcategoria
FROM tbl_producto as tp
  LEFT JOIN tbl_nueva_subcategoria as tnc
    ON tnc.id_subcategoria = tp.id_nueva_subcategoria"""
    df_productos = pd.read_sql(query, engine)
    print('Catálogo obtenido')
except Exception as e:
    print("Error al ejecutar la consulta:", e)


engine.dispose()

Conexión exitosa 🚀
Catálogo obtenido


In [ ]:
df_productos.sample()

,csku,cnombre,cdescripcion,cmarca,id_categoria,categoria
14929,DS-MDS005-E,Expansión para Estación de Carga y Respaldo de...,Expansión para Estación de Carga y Respaldo de...,Hikvision,2.0,Video vigilancia


## Filtrar a No Categorizados

In [ ]:
df_productos = df_productos[df_productos['subcategoria'].isna()]

In [ ]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22015 entries, 2 to 71349
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   csku          22015 non-null  object 
 1   cnombre       22015 non-null  object 
 2   cdescripcion  22015 non-null  object 
 3   cmarca        22012 non-null  object 
 4   id_categoria  0 non-null      float64
 5   categoria     0 non-null      object 
dtypes: float64(1), object(5)
memory usage: 1.2+ MB


## Normalizar Texto 

In [ ]:
# -------------------
# 4. Unir columnas de texto para el modelo
# -------------------
df_productos["texto"] = (
    df_productos["cnombre"].fillna("").astype(str) + " " +
    df_productos["cdescripcion"].fillna("").astype(str) + " " +
    df_productos["cmarca"].fillna("").astype(str)
)

In [ ]:
# Normalizar
df_productos["texto_norm"] = df_productos["texto"].apply(normalize_text)

## Tokenizar y Predecir

In [ ]:
# -------------------
# 5. Tokenizar y predecir
# -------------------
secuencias = tokenizer.texts_to_sequences(df_productos["texto_norm"])
X = pad_sequences(secuencias, maxlen=200)

y_pred = model.predict(X, verbose=0)
categorias_pred = le.inverse_transform(np.argmax(y_pred, axis=1))

## Guardar Resultado

In [37]:
# -------------------
# 6. Guardar resultado en el DataFrame
# -------------------
df_productos["categoria_predicha"] = categorias_pred


In [38]:
df_productos.sample(5)

,csku,cnombre,cdescripcion,cmarca,id_categoria,categoria,texto,texto_norm,categoria_predicha
810,107B-USB-HDMI,"ADAPTADOR DE USB 3.0 A HDMI DOBLE, USB A/C A 2...","ADAPTADOR DE USB 3.0 A HDMI DOBLE, USB A/C A 2...",Startech.Com,NaN,None,"ADAPTADOR DE USB 3.0 A HDMI DOBLE, USB A/C A 2...",adaptador de usb 30 a hdmi doble usb ac a 2 pa...,Computo
43604,905-210,Barra Multicontactos Steren 1.7m 15A 6 Contact...,Barra Multicontactos Steren 1.7m 15A 6 Contact...,Steren,NaN,None,Barra Multicontactos Steren 1.7m 15A 6 Contact...,barra multicontactos steren 17m 15a 6 contacto...,Computo
20543,B38794,"Adaptador magnético de 1/4"" hexagonal para tal...","Adaptador magnético de 1/4"" hexagonal para tal...",Makita,NaN,None,"Adaptador magnético de 1/4"" hexagonal para tal...",adaptador magnetico de 14 hexagonal para talad...,Otros
28830,RE7504,"Pantalla BenQ RE7504 Interactiva 75"" 4K UHD 3...","Pantalla BenQ RE7504 Interactiva 75"" 4K UHD 3...",Benq,NaN,None,"Pantalla BenQ RE7504 Interactiva 75"" 4K UHD 3...",pantalla benq re7504 interactiva 75 4k uhd 384...,Computo
62773,LP-KJ-516-TTGY,"Módulo Jack Keystone Cat5e (toolless), con ter...","Módulo Jack Keystone Cat5e (toolless), con ter...",LinkedPRO,NaN,None,"Módulo Jack Keystone Cat5e (toolless), con ter...",modulo jack keystone cat5e toolless con termin...,Video vigilancia


In [39]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22015 entries, 2 to 71349
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   csku                22015 non-null  object 
 1   cnombre             22015 non-null  object 
 2   cdescripcion        22015 non-null  object 
 3   cmarca              22012 non-null  object 
 4   id_categoria        0 non-null      float64
 5   categoria           0 non-null      object 
 6   texto               22015 non-null  object 
 7   texto_norm          22015 non-null  object 
 8   categoria_predicha  22015 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.7+ MB


## Actualizar Info

In [40]:
# Conexión a la base de datos
engine = conection_bd()

Conexión exitosa 🚀


In [41]:
# Prepara los datos: (categoria_predicha, csku)
updates = list(zip(df_productos['categoria_predicha'], df_productos['csku']))

In [42]:
# Limpieza opcional de caracteres nulos
updates = [(cat.replace('\u0000', '') if isinstance(cat, str) else cat, sku) for cat, sku in updates]

In [43]:
# Conexión cruda para psycopg2
conn = engine.raw_connection()
cursor = conn.cursor()

In [44]:
# Consulta SQL para actualizar
query = """
UPDATE tbl_producto
SET categoria_predicha = %s
WHERE csku = %s;
"""

In [45]:
# Ejecuta en lote para mejorar rendimiento
execute_batch(cursor, query, updates)

In [46]:
# Confirmar cambios
conn.commit()

# Cerrar conexiones
cursor.close()
conn.close()
engine.dispose()